**Note:** Please run the code in VS Code for the correct output and wait for some time to get the result.


In [1]:
import vtk
import ipywidgets as widgets
from IPython.display import display, clear_output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# function to load .vti file
def load_vti_data(file_path):
    data_reader = vtk.vtkXMLImageDataReader()
    data_reader.SetFileName(file_path)
    data_reader.Update()
    return data_reader.GetOutput()

# Reading the .vti file using VTK reader
# This file contains 3D scalar field data for visualization
turbulence_data = load_vti_data('mixture.vti')

# Extracting scalar field values from the dataset
total_points = turbulence_data.GetNumberOfPoints()
scalar_array = turbulence_data.GetPointData().GetArray('ImageFile')

# Extracting coordinates and scalar values
coords = np.array([turbulence_data.GetPoint(i) for i in range(total_points)])
scalars = np.array([scalar_array.GetValue(i) for i in range(total_points)])

data_dict = {'x': coords[:, 0], 'y': coords[:, 1], 'z': coords[:, 2], 'scalar': scalars}

# Defining initial visualization parameters
chosen_colorscale = 'plasma'
initial_isovalue = 0.0

# Creating the initial 3D isosurface plot
isosurface_plot = go.Figure(data=go.Isosurface(
    x=data_dict['x'], y=data_dict['y'], z=data_dict['z'], value=data_dict['scalar'],
    # isomin and isomax are set to initial isovalue for correct isosurface generation
    isomin=initial_isovalue, isomax=initial_isovalue,
    surface_count=1, showscale=False,
    colorscale=chosen_colorscale,
    # cmin and cmax are set to min and max scalar values to show correct color when isovalue changes
    cmin=min(data_dict['scalar']), cmax=max(data_dict['scalar'])
))

# Creating the initial histogram plot to visualize scalar distribution
# nbinsx is set to different value but to match image in assignment.pdf, value 30 is chosen
histogram_plot = go.Figure(data=[go.Histogram(x=data_dict['scalar'], nbinsx=30)])
histogram_plot.update_layout(
    xaxis_title="Vortex scalar values", yaxis_title="Frequency",
    xaxis=dict(visible=True), yaxis=dict(visible=True)
)
histogram_plot.update_traces(marker_color='blue', opacity=0.65)

# Creating slider widget for selecting isovalue dynamically
isovalue_slider = widgets.FloatSlider(
    value=initial_isovalue,
    min=min(data_dict['scalar']), max=max(data_dict['scalar']),
    step=0.01,
    description='Isoval:', continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.2f',
    layout=widgets.Layout(width='600px')
)

# Creating reset button to reset isovalue to 0.0
reset_button = widgets.Button(description="Reset", button_style='primary')

# reset flag to know if the reset button was clicked
reset_flag = True

# Function to update the isosurface plot based on the selected isovalue
def update_isosurface(isovalue):
    isosurface_plot.update_traces(isomin=isovalue, isomax=isovalue)

# Function to update the histogram plot based on the selected isovalue
def update_histogram(isovalue):
    global reset_flag
    if reset_flag and isovalue == 0.00:
        reset_flag = False
        filtered_values = data_dict['scalar']
    else:
        # Filter values within a range around isovalue
        filtered_values = list(filter(lambda v: isovalue - 0.25 <= v <= isovalue + 0.25, data_dict['scalar']))
    
    # Update histogram with filtered values
    if len(histogram_plot.data) > 0:
        histogram_plot.data[0].x = filtered_values
    else:
        histogram_plot.add_trace(go.Histogram(x=filtered_values, nbinsx=30))

    histogram_plot.update_layout(
        xaxis_title="Vortex scalar values", yaxis_title="Frequency",
        xaxis=dict(visible=True), yaxis=dict(visible=True)
    )

# Function to display the combined isosurface and histogram plots
def show_combined_plot():
    fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'scene'}, {'type': 'histogram'}]])
    fig.add_trace(isosurface_plot.data[0], row=1, col=1)
    fig.add_trace(histogram_plot.data[0], row=1, col=2)
    fig.update_layout(
        height=500, width=900,
        xaxis_title="Vortex scalar values",
        yaxis_title="Frequency",
        scene=dict(
            xaxis=dict(showticklabels=False),
            yaxis=dict(showticklabels=False),
            zaxis=dict(showticklabels=False)
        )
    )
    return fig

# Function to handle slider changes
def on_slider_change(changed_value):
    update_isosurface(changed_value.new)
    update_histogram(changed_value.new)
    with plots_output:
        clear_output(wait=True)
        display(show_combined_plot())

# Function to reset isovalue to its default
def reset_clicked():
    global reset_flag
    reset_flag = True
    isovalue_slider.value = initial_isovalue  # Resets the slider to 0.0

# Binding reset button to reset function
reset_button.on_click(lambda _: reset_clicked())

# Binding slider value change to update functions
isovalue_slider.observe(on_slider_change, names='value')

# Creating a container which holds slider and reset button
controls_container = widgets.HBox([isovalue_slider, reset_button])

# Creating an output widget to show the combined plots
plots_output = widgets.Output()

# Creating a main container to place controls and plots vertically
main_container = widgets.VBox([controls_container, plots_output])

# Displaying the main container
display(main_container)

# Dipslaying the initial combined plots
with plots_output:
    display(show_combined_plot())